Goal

predict_cell_index with set_of_csi

In [1]:
import numpy as np
import random
import math

In [11]:
def set_of_csi_to_map_data():
    pass
def set_of_csi_to_cell_index():
    pass
def concat_and_flatten_pair_of_set_of_csi(csi_datas_x,csi_datas_y):#might be good for still object
    csi_datas_x=np.array([np.array(csi) for csi in csi_datas_x])
    csi_datas_y=np.array([np.array(csi) for csi in csi_datas_y])

    a=np.array([csi_datas_x,csi_datas_y])
    features=a.reshape(-1)/128
    # print(type(features[0]))
    # features=np.concatenate(a,axis=0)
    return features
def sd_on_pair_of_set_csi_then_concat_and_flatten(csi_datas_x,csi_datas_y):#might be good for moving object
    if len(csi_datas_x)!=len(csi_datas_y):
        return ValueError(f"len(csi_datas_x)!=len(csi_datas_y) ; {len(csi_datas_x)}!={len(csi_datas_y)}")
    #mean of columns for both csi
    mean_of_columns_x=np.mean(csi_datas_x, axis=0)
    mean_of_columns_y=np.mean(csi_datas_y, axis=0)

    row_sampling_range=20 #not used
    rows_amount=min(len(csi_datas_x),len(csi_datas_y)) #likely equal
    transformed_x=[[0 for __ in range(128)] for _ in range(rows_amount-row_sampling_range)]
    transformed_y=[[0 for __ in range(128)] for _ in range(rows_amount-row_sampling_range)]
    
    for row_index in range(rows_amount):
        for column_index in range(128):
            new_val_x=0
            new_val_y=0
            for row_index_sub in range(row_index,row_index+row_sampling_range):
                new_val_x+=(csi_datas_x[row_index_sub][column_index]-mean_of_columns_x[column_index])**2
                new_val_y+=(csi_datas_y[row_index_sub][column_index]-mean_of_columns_y[column_index])**2
            new_val_x/=row_sampling_range
            new_val_y/=row_sampling_range
            new_val_x=math.sqrt(new_val_x)
            new_val_y=math.sqrt(new_val_y)
            transformed_x[row_index][column_index]=new_val_x
            transformed_y[row_index][column_index]=new_val_y
    features=np.array([transformed_x,transformed_y])
    # features=np.array(r).reshape(2,-1)
    features=features.reshape(-1)
    return features
def pair_of_set_of_csi_to_features(csi_datas_x,csi_datas_y):
    # return sd_on_pair_of_set_csi_then_concat_and_flatten(csi_datas_x,csi_datas_y)
    return concat_and_flatten_pair_of_set_of_csi(csi_datas_x,csi_datas_y)

In [12]:
#test

csi_datas_x = [

]
for i in range(0,20):
    csi_datas_x.append([random.randint(-127,128) for i in range(128)])
csi_datas_y = [

]
for i in range(0,20):
    csi_datas_y.append([random.randint(-127,128) for i in range(128)])

r=pair_of_set_of_csi_to_features(csi_datas_x,csi_datas_y)
print(r.shape)
# r=pair_of_set_of_csi_to_features(csi_datas_x,csi_datas_y)

print(r)


(5120,)
[ 0.109375  -0.65625    0.03125   ...  0.0546875 -0.4375     0.2265625]


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import scipy.io as sio
import os
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.datasets import load_iris 
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [14]:
folder_dir="../CSI_Data_06032024"
horizontal="Horizontal"
vertical="Vertical"
X=[]
Y=[]

In [15]:
cell_index=0
show_heatmap=0;
sampling_range=20
X=[]
Y=[]
def csi_data_str_to_list(s):
    s=s.replace("[","").replace("]","").strip().split(" ")
    a=list(map(int,s))
    return a
for cell_index in range(0,16+1):
    cell_name=str(cell_index).zfill(2)
    df_hor=pd.read_csv(f"{folder_dir}/{horizontal}/{cell_name}_16.csv")
    df_ver=pd.read_csv(f"{folder_dir}/{vertical}/{cell_name}_16.csv")
    df_hor["CSI_DATA"]=df_hor["CSI_DATA"].apply(csi_data_str_to_list)
    df_ver["CSI_DATA"]=df_ver["CSI_DATA"].apply(csi_data_str_to_list)


    if(len(df_hor)!=len(df_ver)):
        print(f"cell{cell_index} Not equal in row together")
    rows_amount=min(len(df_hor),len(df_ver))
    sampling_range=20


    # print(df_hor["CSI_DATA"].values)
    for row_index in range(0,rows_amount-sampling_range):
    # for row_index in range(0,5):
        csi_datas_x=df_hor["CSI_DATA"].values[row_index:row_index+sampling_range]
        csi_datas_y=df_hor["CSI_DATA"].values[row_index:row_index+sampling_range]
        # print(pair_of_set_of_csi_to_features(csi_datas_x,csi_datas_y))
        X.append(pair_of_set_of_csi_to_features(csi_datas_x,csi_datas_y))
        Y.append(cell_index)
        # break
    # break
        

cell0 Not equal in row together
cell1 Not equal in row together
cell2 Not equal in row together
cell3 Not equal in row together
cell4 Not equal in row together
cell5 Not equal in row together
cell6 Not equal in row together
cell7 Not equal in row together
cell8 Not equal in row together
cell9 Not equal in row together
cell10 Not equal in row together
cell11 Not equal in row together
cell12 Not equal in row together
cell13 Not equal in row together
cell14 Not equal in row together
cell15 Not equal in row together
cell16 Not equal in row together


In [16]:
X=np.array(X)
print(X.shape)
Y=np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


(80861, 5120)


In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_train[0])
print(y_train)


(64688, 5120)
(64688,)
[ 0.65625 -0.5      0.03125 ...  0.       0.       0.     ]
[ 0  4 13 ... 16  0  3]


MLP model

In [18]:
print("Training")
mlp_model=MLPClassifier(hidden_layer_sizes=(64,64,64),max_iter=1000,random_state=42)
mlp_model.fit(X_train, y_train)

Training


c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(64, 64, 64), max_iter=1000, random_state=42)

In [19]:
mlp_model.score(X_test, y_test)

0.41847523650528656

In [22]:
import joblib
joblib.dump(mlp_model,"mlp_model.pkl")

['mlp_model.pkl']

In [53]:
from sklearn.preprocessing import OneHotEncoder
import joblib
encoder=OneHotEncoder()
y_encoded=encoder.fit_transform(Y.reshape(-1,1)).toarray()

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [59]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(5120,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train_encoded, epochs=100, batch_size=5, verbose=1)


Epoch 1/100


ValueError: in user code:

    File "c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\yanot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    F

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import joblib
encoder=OneHotEncoder(sparse=False)
y_encoded=encoder.fit_transform(y_train.reshape(-1,1))